In [2]:
import requests
from bs4 import BeautifulSoup
import re
import urllib.parse
import tqdm
import json
import os

def get_html(url: str):
    # Returns bs4.BeautifulSoup object from url using Requests
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')
    return soup

def get_mix_title(soup):
    # Returns title of show from bs4 object containing NTS 'shows' html
    return soup.find("h1").text

def format_name(name: str):
    # format name - for now just removed commas if last character
    if ',' in name:
        name = name.replace(',','')
    return name

def get_tracklist(soup):
    # Returns list containing basic track metadata (artist, track, url and episode number from bs4 object containing NTS 'episodes' html
    tracklist = []
    for element in soup.find_all("li"):
        artist = element.find("span",{"class":"track__artist"}).text
        track = element.find("span",{"class":"track__title"}).text

        tracklist.append({"artist": format_name(artist),
                          "track" : track})
    return tracklist

def is_valid_nts_url(url):
    """
    Validate if URL is a valid NTS URL
    """
    try:
        parsed = urllib.parse.urlparse(url)
        # check if the URL is well-formed
        if not all([parsed.scheme, parsed.netloc, parsed.path]):
            return False
        # check if the URL is https and domain is 'www.nts.live'
        if parsed.scheme != 'https' or parsed.netloc != 'www.nts.live':
            return False
        # check if URL path follows the expected structure
        if not re.match(r'^/shows/.+/episodes/.+$', parsed.path):
            return False
        return True
    except ValueError:
        return False

def get_mix_title_from_url(url: str):
    """
    Extract the show title from a NTS URL
    """
    # split the url by slashes and get the last part as the title
    title = url.split("/")[-1]

    return title

# Check if the program was interrupted previously by checking if the 'last_processed_line.txt' file exists
if os.path.exists('data/last_processed_line.txt'):
    with open('data/last_processed_line.txt', 'r') as f:
        start_line = int(f.read())
else:
    start_line = 0

with open('data/links.txt', 'r') as f:
    lines = f.readlines()

valid_urls = [line.strip() for line in lines if is_valid_nts_url(line.strip())]

with open('data/tracklists.txt', 'a') as f:  # use append mode in case of interruption
    for i, url in enumerate(tqdm.tqdm(valid_urls[start_line:])):
        try:
            html = get_html(url)
            title = get_mix_title_from_url(url)
            pretty_title = get_mix_title(html)
            tracklist = get_tracklist(html)
            f.write(json.dumps({"title": title, "pretty-title": pretty_title, "tracklist": tracklist}) + '\n')  

            # Write the last processed line number to a file
            with open('data/last_processed_line.txt', 'w') as lpl:
                lpl.write(str(start_line + i))
        except Exception as e:
            print(f'Error processing URL {url}: {e}')
            continue


 12%|█▏        | 3410/28919 [13:32<1:33:19,  4.56it/s]

Error processing URL https://www.nts.live/shows/babastiltz/episodes/babastiltz-invalid%20date: 'NoneType' object has no attribute 'text'


 19%|█▊        | 5355/28919 [21:05<1:26:16,  4.55it/s]

Error processing URL Https://www.nts.live/shows/guests/episodes/kẹo-keo-7th-november-2022: 'NoneType' object has no attribute 'text'


 19%|█▊        | 5394/28919 [21:13<1:24:41,  4.63it/s]

...

 97%|█████████▋| 28024/28919 [1:46:34<03:31,  4.24it/s]

Error processing URL https://www.nts.live/shows/guests/episodes/bamao-yenda-9th-april-2018: 'NoneType' object has no attribute 'text'


 99%|█████████▉| 28610/28919 [1:48:52<01:13,  4.20it/s]

Error processing URL https://www.nts.live/shows/naomi/episodes/naomi-hello-dj-13th-december-2017: 'NoneType' object has no attribute 'text'


100%|██████████| 28919/28919 [1:50:07<00:00,  4.38it/s]
